In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
import seaborn as sns
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report, accuracy_score, f1_score
import itertools

In [2]:
#file_path = r"C:\Users\jens.nilsen\OneDrive - Bouvet Norge AS\Documents\GitHub\trfkaipoc\data_2022-2025.csv"
file_path = r"C:\Users\jens.nilsen\OneDrive - Bouvet Norge AS\Documents\GitHub\trfkaipoc\data_2022-2025_norge.csv"
df = pd.read_csv(file_path, sep=";")

In [3]:
df = df[df["EGS.VEDTAK.10670"].notna()]

In [4]:
df['Avslag_ind'] = df['EGS.VEDTAK.10670'].apply(lambda x: 1 if x == "Avslag" else 0)

In [5]:
if "Kurvatur, horisontalelement" in df.columns:
    df["Kurvatur, horisontal"] = df["Kurvatur, horisontalelement"]

In [6]:
features = [
    'Avslag_ind',
    "ÅDT, total",
    "ÅDT, andel lange kjøretøy",
    "Fartsgrense",
    "Avkjørsel, holdningsklasse",
    "Funksjonsklasse",
    "Avkjørsler",
    "Trafikkulykker",
    "EGS.BRUKSOMRÅDE.1256", 
    "Kurvatur, horisontal", 
    "Kurvatur, stigning"
]

# Encode categorical features
df_encoded = pd.get_dummies(df[features])

In [7]:
df_encoded=df_encoded.dropna()

In [8]:
df_encoded['sving_ind'] = np.where(df_encoded['Kurvatur, horisontal'].abs() > 99000, 0, 1)
df_encoded['sving'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, df_encoded['Kurvatur, horisontal'].abs(), 0)
df_encoded['bakke']=df_encoded['Kurvatur, stigning'].abs()
df_encoded['sving_sigmoid'] = np.where(df_encoded['Kurvatur, horisontal'].abs() < 99000, 1/(1+np.exp(-0.001*df_encoded['Kurvatur, horisontal'].abs())), 0)
df_encoded['antall_lange_kj']=df_encoded['ÅDT, total']*df_encoded['ÅDT, andel lange kjøretøy']/100
df_encoded = df_encoded.drop(['Kurvatur, horisontal', 'Kurvatur, stigning'], axis=1)
y = df_encoded['Avslag_ind']        # target
X = df_encoded.drop(columns=['Avslag_ind'])  # all other columns

In [9]:
poly = PolynomialFeatures(3, include_bias=False, interaction_only=True) 
X = pd.DataFrame(
    poly.fit_transform(X),
    columns=poly.get_feature_names_out(X.columns)
)

In [10]:
# Identify binary dummy columns: only {0,1} or {0.0,1.0}
binary_cols = [
    col for col in X.columns
    if np.isin(X[col].dropna().unique(), [0,1]).all()
]

continuous_cols = [col for col in X.columns if col not in binary_cols]

# Transform only continuous columns
scaler = PowerTransformer()
X_cont_scaled = pd.DataFrame(
    scaler.fit_transform(X[continuous_cols]),
    columns=continuous_cols,
    index=X.index
)

# Combine back into full feature matrix
X = pd.concat([X_cont_scaled, X[binary_cols]], axis=1)
X = X.sample(frac=1, random_state=42) 

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=43)

In [12]:
model = RandomForestClassifier(
    n_estimators=1000,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# Get feature importances
importances = pd.Series(model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)
importances
# Select top 10 features
top_features = importances_sorted.index[:10]


In [13]:
X_train = X_train[top_features]

In [14]:
X_val=X_val[top_features]

In [15]:
X_val

,"ÅDT, total ÅDT, andel lange kjøretøy Fartsgrense",Fartsgrense antall_lange_kj,"ÅDT, andel lange kjøretøy Fartsgrense antall_lange_kj","ÅDT, total ÅDT, andel lange kjøretøy","ÅDT, total Fartsgrense bakke","ÅDT, andel lange kjøretøy antall_lange_kj","ÅDT, andel lange kjøretøy Fartsgrense",antall_lange_kj,"ÅDT, total Funksjonsklasse_E - Lokale adkomstveger antall_lange_kj",Fartsgrense Avkjørsler antall_lange_kj
539,0.756755,0.756924,1.091119,0.544959,-1.356711,0.914871,1.637396,0.552381,-0.737561,0.604043
500,1.323944,1.323991,1.631423,1.109654,-0.605126,1.457432,1.761921,1.115524,-0.737561,-0.047567
220,-0.424740,-0.424763,-0.666558,-0.205989,0.041557,-0.479848,-1.210773,-0.205821,-0.737561,0.241900
327,-0.026576,-0.026484,-0.310941,0.188734,1.421584,-0.120343,-1.210773,0.193707,1.497995,0.740919
25,-0.116261,-0.116191,-0.021728,-0.066844,-0.033214,0.021572,0.094461,-0.064803,-0.737561,-1.796742
...,...,...,...,...,...,...,...,...,...,...
647,0.901895,0.902048,0.521021,1.292883,1.828443,0.890643,-1.659300,1.296546,1.597272,1.323452
316,-0.681983,-0.682091,-0.686254,-0.907448,-0.325597,-0.899223,0.277017,-0.916646,1.322605,0.208742
505,1.557032,1.556977,1.214151,1.454051,-0.615017,1.142748,-0.247941,1.455016,-0.737561,1.278301
515,1.221049,1.221130,1.451007,1.007549,1.024507,1.276765,1.505730,1.014268,-0.737561,0.584638


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=43)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import PredefinedSplit

# Define the parameter distributions
param_dist = {
    'n_estimators': randint(100, 500),
     'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 25),
    'sampling_strategy': uniform(0.1, 0.6)
    #'max_leaf_nodes': randint(10, 100)
}

model = BalancedRandomForestClassifier(random_state=42, n_jobs=-1, replacement=True, bootstrap=True)

# Combine training + validation for PredefinedSplit
X_combined = np.vstack([X_train, X_test])
y_combined = np.concatenate([y_train, y_test])

# -1 for training rows, 0 for validation rows
test_fold = [-1] * len(X_train) + [0] * len(X_test)
ps = PredefinedSplit(test_fold)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=1000,                  # number of random combinations to try
    cv=ps,
    #cv=4,
    scoring="f1_weighted",
    n_jobs=1,
    verbose=1,
    random_state=42
)

random_search.fit(X_combined, y_combined)
#random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)


Fitting 1 folds for each of 1000 candidates, totalling 1000 fits


In [ ]:
y_pred=random_search.predict(X_val)
y_proba=random_search.predict_proba(X_val)[:,1]

In [ ]:
len(y_pred)

In [ ]:
print(classification_report(y_val, y_pred, digits=4))

In [ ]:
df_plot = pd.DataFrame({'y_val': y_val, 'y_proba': y_proba})

# Plot distributions
plt.figure(figsize=(8,5))
sns.kdeplot(data=df_plot[df_plot['y_val'] == 0]['y_proba'], label='Actual class 0', fill=True)
sns.kdeplot(data=df_plot[df_plot['y_val'] == 1]['y_proba'], label='Actual class 1', fill=True)
plt.title("Distribution of predicted probabilities by actual class")
plt.xlabel("Predicted probability (positive class)")
plt.ylabel("Density")
plt.legend()

In [ ]:
model = random_search.best_estimator_

In [ ]:
importances = pd.Series(model.feature_importances_, index=X_train.columns)
importances_sorted = importances.sort_values(ascending=False)

In [ ]:
importances_sorted

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Best model (already trained on exactly these 2 features)
best_model = random_search.best_estimator_

# Use top 2 features
top_features = importances_sorted.index[:2]
X_top_train = X_train[top_features]
X_top_test  = X_test[top_features]

y_test_reset = y_test.reset_index(drop=True)

# Create mesh grid
x_min, x_max = X_top_train[top_features[0]].min() - 1, X_top_train[top_features[0]].max() + 1
y_min, y_max = X_top_train[top_features[1]].min() - 1, X_top_train[top_features[1]].max() + 1

xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 400),
    np.linspace(y_min, y_max, 400)
)

# Predict grid
Z = best_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Color maps (subtle green background, strong red)
cmap_light = ListedColormap(["#bbffbb", "#ff9999"])  # green background for 0, red for 1
cmap_bold  = ListedColormap(["green", "red"])

plt.figure(figsize=(9,7))

# Plot decision region
plt.contourf(xx, yy, Z, alpha=0.35, cmap=cmap_light)

# Plot test points (circles)
plt.scatter(
    X_top_test[top_features[0]],
    X_top_test[top_features[1]],
    c=y_test_reset,
    cmap=cmap_bold,
    edgecolor='black',
    s=90,
    marker='o',            # round points for real data
    label='Test points'
)

plt.xlabel(top_features[0])
plt.ylabel(top_features[1])
plt.title("2D Decision Boundary with Test Set Points")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Extract best model and top 3 features
best_model = random_search.best_estimator_
top_features = importances_sorted.index[:3]

# Use TEST set only for real points
X_top_test = X_test[top_features].reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

# 3D grid bounds based on TRAIN (model space still valid)
grid_size = 25
x_min, x_max = X_train[top_features[0]].min() - 1, X_train[top_features[0]].max() + 1
y_min, y_max = X_train[top_features[1]].min() - 1, X_train[top_features[1]].max() + 1
z_min, z_max = X_train[top_features[2]].min() - 1, X_train[top_features[2]].max() + 1

xx, yy, zz = np.meshgrid(
    np.linspace(x_min, x_max, grid_size),
    np.linspace(y_min, y_max, grid_size),
    np.linspace(z_min, z_max, grid_size)
)

grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]

# Predict class decision
Z = random_search.predict(grid_points).ravel()

# Separate + and − region points
pos_points = grid_points[Z == 1]
neg_points = grid_points[Z == 0]

fig = go.Figure()

# --- Decision region markers (diamonds) ---
fig.add_trace(go.Scatter3d(
    x=pos_points[:,0], y=pos_points[:,1], z=pos_points[:,2],
    mode='markers',
    marker=dict(
        size=6,
        symbol="diamond",
        color="red",
        opacity=0.35
    ),
    name='Positive Region'
))

fig.add_trace(go.Scatter3d(
    x=neg_points[:,0], y=neg_points[:,1], z=neg_points[:,2],
    mode='markers',
    marker=dict(
        size=4,
        symbol="diamond",
        color="green",
        opacity=0.10
    ),
    name='Negative Region'
))

# --- TEST data points (real observations) ---
fig.add_trace(go.Scatter3d(
    x=X_top_test[top_features[0]],
    y=X_top_test[top_features[1]],
    z=X_top_test[top_features[2]],
    mode='markers',
    marker=dict(
        size=7,
        symbol="circle",            # Different shape
        color=np.where(y_test_reset == 1, "red", "green"),
        line=dict(width=1, color='black')
    ),
    name='Test set points'
))

fig.update_layout(
    scene=dict(
        xaxis_title=top_features[0],
        yaxis_title=top_features[1],
        zaxis_title=top_features[2],
    ),
    title="3D Decision Region (Diamonds) with Test Set Points (Circles)",
    width=950,
    height=750
)

fig.show()


In [ ]:
Z.sum()